# Import Some Packages

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn

import pandas as pd
import numpy as np

import csv
import os
from tqdm import tqdm

from torch.utils.tensorboard import SummaryWriter

import math

# some utilties

In [2]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# configurations

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'valid_ratio' : 0.2,
    'seed':5121014,
    'select_all':True,
    'batch_size':256,
    'learning_rate':1e-3,
    'n_epochs':3000,
    'save_path':'./models/model.ckpt',
    'early_stop':400
}

# 数据模块
## 划分数据集

In [4]:
def train_valid_set(data_set, valid_ratio, seed):
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

## Dataset

In [5]:
class convid19Dataset(Dataset):
    def __init__(self,x,y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)
    
    def __getitem__(self,idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx],self.y[idx]
    
    def __len__(self):
        return len(self.x)

## 选择特征值

In [6]:
def select_feather(train_data,valid_data,test_data,select_all = True):
    y_train,y_valid = train_data[:,-1],valid_data[:,-1]
    raw_x_train,raw_x_valid,raw_x_test = train_data[:,1:-1],valid_data[:,1:-1],test_data[:,1:]
    
    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        feat_idx = [] #待选择
    return raw_x_train[:,feat_idx],raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid

## DataLoader

In [7]:
same_seed(config['seed'])
train_data,test_data = pd.read_csv('./covid.train.csv').values,pd.read_csv('./covid.test.csv').values
train_data,valid_data = train_valid_set(train_data,config['valid_ratio'],config['seed'])

print(f'''train_data_size:{train_data.shape}
valid_data_size:{valid_data.shape}
test_data_size:{test_data.shape}''')
x_train,x_valid,x_test,y_train,y_valid = select_feather(train_data,valid_data,test_data,config['select_all'])
print(f"number of feathers:{x_train.shape}")

train_dataset,valid_dataset,test_dataset = convid19Dataset(x_train,y_train),\
                                        convid19Dataset(x_valid,y_valid),\
                                        convid19Dataset(x_test)
train_dataloader = DataLoader(train_dataset,batch_size = config['batch_size'],shuffle = True,pin_memory = True)
valid_dataloader = DataLoader(valid_dataset,batch_size = config['batch_size'],shuffle = True,pin_memory = True)
test_dataloader = DataLoader(test_dataset,batch_size = config['batch_size'],shuffle = False,pin_memory = True)


train_data_size:(2160, 95)
valid_data_size:(540, 95)
test_data_size:(893, 94)
number of feathers:(2160, 93)


# Neural Network Model

In [8]:
class My_model(nn.Module):
    def __init__(self,input_dim):
        super(My_model,self).__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(input_dim,32),
            nn.ReLU(),
            nn.Linear(32,16),
            nn.ReLU(),
            nn.Linear(16,8),
            nn.ReLU(),
            nn.Linear(8,1),
        )
    def forward(self,x):
        x = self.layers(x)
        x = x.squeeze(1)
        return x

# Training Loop

In [9]:
def trainer(train_loader,valid_loader,model,config,device):
    criterion = nn.MSELoss(reduction = 'mean')
    
    optimizer = torch.optim.Adam(model.parameters(),lr = config['learning_rate'])
    
    writer = SummaryWriter()
    
    if not os.path.isdir('./models'):
        os.mkdir('./models')
    n_epochs,best_loss,step,early_stop_count = config['n_epochs'],math.inf,0,0
    
    for epoch in range(n_epochs):
        model.train()
        loss_record = []
        train_pbar = tqdm(train_loader,leave = True,position = 0)
        for x,y in train_pbar:
            optimizer.zero_grad()
            x,y = x.to(device),y.to(device)
            pred = model(x)
            loss = criterion(pred,y)
            loss.backward()
            optimizer.step()
            step+=1
            loss_record.append(loss.detach().item())
            
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss':loss.detach().item()})
        
        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train',mean_train_loss,step)

        model.eval()
        loss_record = []
        for x,y in valid_loader:
            x,y = x.to(device),y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred,y)
            loss_record.append(loss.item())
        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid',mean_valid_loss,step)
        
        if mean_valid_loss <best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(),config['save_path'])
            print('Saving model with loss {:.3f}'.format(best_loss))
            early_stop_count = 0
        else:
            early_stop_count += 1
        
        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

# Start training!

In [10]:
model = My_model(input_dim = x_train.shape[1]).to(device)
trainer(train_dataloader,valid_dataloader,model,config,device)

Epoch [1/3000]: 100%|██████████████████████████████████████████████████████████| 9/9 [00:03<00:00,  2.73it/s, loss=188]


Epoch [1/3000]: Train loss: 255.4145, Valid loss: 197.2637
Saving model with loss 197.264


Epoch [2/3000]: 100%|█████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 265.58it/s, loss=110]


Epoch [2/3000]: Train loss: 142.9941, Valid loss: 80.6042
Saving model with loss 80.604


Epoch [3/3000]: 100%|████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 250.84it/s, loss=47.4]


...

Epoch [1761/3000]: Train loss: 0.7805, Valid loss: 1.0371


Epoch [1762/3000]: 100%|████████████████████████████████████████████████████| 9/9 [00:00<00:00, 225.75it/s, loss=0.854]


Epoch [1762/3000]: Train loss: 0.7727, Valid loss: 1.0211


Epoch [1763/3000]: 100%|█████████████████████████████████████████████████████| 9/9 [00:00<00:00, 97.89it/s, loss=0.775]

Epoch [1763/3000]: Train loss: 0.7534, Valid loss: 1.1659

Model is not improving, so we halt the training session.


# Plot learning curves with tensorboard

In [11]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

Reusing TensorBoard on port 6006 (pid 4200), started 6 days, 6:51:15 ago. (Use '!kill 4200' to kill it.)

In [12]:
def save_pred(preds,file):
    with open(file,'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id','tested_positive'])
        for i,p in enumerate(preds):
            writer.writerow([i,p])

def predict(test_loader,model,device):
    model.eval()
    preds=[]
    for x in tqdm(test_loader):
        x = x.to(device)
        with torch.no_grad():
            pred = model(x)
            preds.append(pred.detach().cpu())
    preds = torch.cat(preds,dim = 0).numpy()
    return preds 
            
model = My_model(input_dim = x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_dataloader,model,device)
save_pred(preds,'pred.csv')

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 723.09it/s]
